# Обучение модели детекции СИЗ

Этот ноутбук обучает модель YOLOv8 на подготовленных данных.


In [ ]:
# Импорт библиотек
from ultralytics import YOLO
import os
from pathlib import Path
import matplotlib.pyplot as plt
import torch

print("✅ Библиотеки загружены")
print(f"CUDA доступна: {torch.cuda.is_available()}")

# Проверяем конфигурацию
config_path = "config/ppe_data.yaml"
if Path(config_path).exists():
    print(f"✅ Конфигурация найдена: {config_path}")
else:
    print(f"❌ Конфигурация не найдена: {config_path}")
    print("Создайте файл config/ppe_data.yaml с классами helmet и vest")


## Обучение модели

Обучаем YOLOv8 на наших данных. Можно настроить параметры в ячейке ниже.


In [ ]:
# Параметры обучения
EPOCHS = 30
IMG_SIZE = 640
BATCH_SIZE = 16  # Уменьшите для CPU
DEVICE = 0 if torch.cuda.is_available() else 'cpu'
PROJECT = "models"
NAME = "ppe_model"

print(f"Параметры обучения:")
print(f"  Эпохи: {EPOCHS}")
print(f"  Размер изображения: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Батч: {BATCH_SIZE}")
print(f"  Устройство: {DEVICE}")
print(f"  Проект: {PROJECT}/{NAME}")


In [ ]:
# Запуск обучения
print("=== НАЧАЛО ОБУЧЕНИЯ ===")
print(f"Используется устройство: {DEVICE}")

# Загружаем модель
model = YOLO("yolov8n.pt")  # Nano версия - быстрая

# Обучаем
results = model.train(
    data=config_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    device=DEVICE,
    project=PROJECT,
    name=NAME,
    batch=BATCH_SIZE,
    patience=10,  # Ранняя остановка
    save=True,
    plots=True,
    verbose=True
)

print("✅ Обучение завершено!")
print(f"Модель сохранена в: models/{NAME}/weights/best.pt")


## Анализ результатов обучения

Просматриваем метрики и графики обучения.


In [ ]:
# Загружаем результаты
from ultralytics.utils.plotting import plot_results

# Путь к результатам
results_path = Path(f"models/{NAME}/results.png")

if results_path.exists():
    print("📊 График обучения:")
    display = plt.imread(str(results_path))
    plt.figure(figsize=(12, 8))
    plt.imshow(display)
    plt.axis('off')
    plt.title("Результаты обучения YOLOv8")
    plt.show()
else:
    print("⚠️  График результатов не найден")

# Метрики лучшей модели
metrics_path = Path(f"models/{NAME}/results.csv")
if metrics_path.exists():
    import pandas as pd
    df = pd.read_csv(metrics_path)
    print("\n📈 Последние метрики:")
    print(df[['epoch', 'train/box_loss', 'val/box_loss', 'mAP50', 'mAP50-95']].tail())
else:
    print("⚠️  Файл метрик не найден")


## Тестирование модели

Проверяем обученную модель на тестовых изображениях.


In [ ]:
# Загружаем лучшую модель
best_model_path = f"models/{NAME}/weights/best.pt"
model = YOLO(best_model_path)

# Тестируем на изображении из val
test_images = list(Path("data/images/val").glob("*.jpg"))
if test_images:
    test_img = test_images[0]
    print(f"Тестируем на: {test_img.name}")
    
    # Детекция
    results = model(test_img, conf=0.5, save=True, project="output", name="test")
    
    # Показываем результат
    result_img = Path(f"output/test/{test_img.name}")
    if result_img.exists():
        display = plt.imread(str(result_img))
        plt.figure(figsize=(10, 8))
        plt.imshow(display)
        plt.axis('off')
        plt.title(f"Детекция на {test_img.name}")
        plt.show()
    else:
        print("⚠️  Результат не сохранен")
else:
    print("❌ Нет тестовых изображений в data/images/val/")
